# seq2seq: Train

Abdulhakim Alnuqaydan, Ali Kadhim, Sergei Gleyzer, Harrison Prosper

July 2021

## Description

Use an encoder/decoder model built using LSTMs to map symbolic mathematical expressions $f(x)$ to their Taylor series expansions to ${\cal O}(x^5)$.

We've heavily borrowed from Charon Guo's excellent tutorial at:

https://charon.me/posts/pytorch/pytorch_seq2seq_1/


### Using Google Colaboratory
If you want to use Google colab then uncomment the instructions in the next cell. When that cell is executed, you'll be directed to a Google sign-in page. Once signed in, copy the validation code into the entry window on this page.

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/AI')

Mounted at /content/gdrive


In [1]:
# symbolic mathematics
import sympy as sp
from sympy import exp, \
    cos, sin, tan, \
    cosh, sinh, tanh, ln

x = sp.Symbol('x')

# array manipulation
import numpy as np
import random as rn
import math
import time

# PyTorch
import torch
import torch.nn as nn
from torch import optim

# custom data prep and loader
from seq2sequtil import loadData, \
Seq2SeqDataPreparer, Seq2SeqDataLoader

# enable pretty printing of symbolic equations
from IPython.display import display
sp.init_printing(use_latex='mathjax')

In [2]:
#BASE = '/content/gdrive/My Drive/AI'
BASE = '.'

Duration of an epoch.

In [3]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Function to count the number of parameters in a model

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Load sequence data

In [11]:
inputs, targets = loadData('data/seq2seq_data_10000.txt')
print(inputs[8000])
print(targets[8000])

0


-sinh(2⋅x)

     3      
  4⋅x       
- ──── - 2⋅x
   3        


2000


        2              
 4 - 9⋅x     ⎛   2    ⎞
ℯ        ⋅cos⎝7⋅x  - 1⎠

...

 4 ⎛                       3              5                         ⎞    3 ⎛  
x ⋅⎝4374⋅tan(3) + 10935⋅tan (3) + 6561⋅tan (3) + 0.00051440329218107⎠ + x ⋅⎝-2

            2             4   ⎞    2 ⎛                  3                     
43 - 972⋅tan (3) - 729⋅tan (3)⎠ + x ⋅⎝81⋅tan(3) + 81⋅tan (3) - 0.0555555555555

   ⎞     ⎛          2   ⎞             
556⎠ + x⋅⎝-9 - 9⋅tan (3)⎠ + tan(3) + 1


8000


             ⎛ 3⎞
 -5⋅x - 1    ⎜x ⎟
ℯ        ⋅cos⎜──⎟
             ⎝3 ⎠

     4  -1        3  -1       2  -1                
625⋅x ⋅ℯ     125⋅x ⋅ℯ     25⋅x ⋅ℯ          -1    -1
────────── - ────────── + ───────── - 5⋅x⋅ℯ   + ℯ  
    24           6            2                    


	cos(3*x**3/9)/exp(5*x+1)

	exp(-1)-5*x*exp(-1)+25*x**2*exp(-1)/2-125*x**3*exp(-1)/6+625*x**4*exp(-1)/24



### Convert sequence data
  1. Scan input and output sequences and construct maps of characters (tokens) to indices, one map for input sequences and another for target sequences.
  1. Pad sequences to the same length
  1. Split into train, validation, and test sets.

In [13]:
fractions=[8/10, 9/10]
db = Seq2SeqDataPreparer(inputs, targets, fractions)
print(db)

number of seq-pairs (train):     8000
number of seq-pairs (valid):     1000
number of seq-pairs (test):      1000

number of source tokens:           31
max source sequence length:        81

number of target tokens:           35
max target sequence length:       883


## Implement encoder

### LSTM 

An LSTM is a function that is typically conceptualized as a "device" containing various "gates" that filter input data in different ways. This creative conceptual reasoning has yielded functions with amazing capabilities. However, it is far from clear that this approach will be the way forward in the future. Why? Because we have compelling evidence that highly creative conceptual reasoning, while impressive, is not, actually, needed to arrive at functions with immense capability. The existence of human brains that have evolved through natural selection is an existence proof that immensely capable functions can be arrived at through trial and error.

No doubt, one day, someone will devise an effective evolutionary algorithm that will compress millions of years of evolution into a matter of weeks or even days in order to construct immensely capable functions that could far outstrip what could be done through creative conceptual reasoning. 

#### LSTM Function

At time step $t$, instances of the LSTM class in PyTorch compute the function

\begin{align*}
 i_t & = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}),\\
 f_t & = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}),\\
 g_t & = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}),\\ \\
 c_t & = f_t \odot c_{t-1} + i_t \odot g_t,\\
 o_t & = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}),\\
 h_t & = o_t \odot \tanh(c_t),
\end{align*}

where $\sigma$ is a sigmoid and $\odot$ is the Hadamard product. The functions $i_t$, and $f_t$ are called the *input* and *forget* gates, respectively, while $c_t$ and $h_t$ are called the *cell* and *hidden* states, respectively, of the LSTM. The cell and hidden states $c_{t-1}$ and $h_{t-1}$ are from the previous time step. The number of elements that comprise the cell and hidden states is specified by the *hidden_size* argument of the PyTorch LSTM. The $W$s and $b$s are the parameters of the LSTM function. The LSTM outputs $o_t, (h_t, c_t)$.

### PyTorch LSTM Parameters

* input_size – The number of expected features in the input x
* hidden_size – The number of features in the hidden state h
* num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
* bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
* batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False
* dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0
* bidirectional – If True, becomes a bidirectional LSTM. Default: False
* proj_size – If > 0, will use LSTM with projections of corresponding size. Default: 0

### Encoder
  * max_seq_len - Maximum length of source, i.e. input, sequences
  * num_features - Number of unique source tokens 

In [14]:
HIDDEN_SIZE = 32
NUM_LAYERS  = 10
# use GPU if available
DEVICE      = torch.device("cuda" \
                           if torch.cuda.is_available() \
                           else "cpu")

class Encoder(nn.Module):
    def __init__(self, max_seq_len, num_features, 
                 embed_size=10, 
                 hidden_size=HIDDEN_SIZE, 
                 num_layers=NUM_LAYERS, 
                 dropout=0.5,
                 device=DEVICE):
        super(Encoder, self).__init__()
        
        self.max_seq_len = max_seq_len  # unused at present
        self.num_features= num_features
        self.embed_size  = embed_size
        self.hidden_size = hidden_size
        self.num_layers  = num_layers
        self.dropout     = dropout
        
        # The embedding will map each of "num_features" indices into 
        # a vector of dimension embed_size, where typically, 
        # embed_size << num_features.
        #
        # The shape of the inputs to embedding is 
        #  (max_seq_len, batch_size).
        #
        # The shape of the outputs is 
        #  (max_seq_len, batch_size, embed_size)
        self.embedding = nn.Embedding(num_features, 
                                      embed_size).to(device)
        
        # By default, LSTM expects the input shape to be
        #  (max_seq_len, batch_size, embed_size)
        self.lstm = nn.LSTM(embed_size, 
                            hidden_size, 
                            num_layers,
                            dropout=dropout).to(device)


    def forward(self, x):
        # x.shape: (max_seq_len, batch_size)
       
        x = self.embedding(x)
        # x.shape: (max_seq_len, batch_size, embed_size)
        
        output, (hidden, cell) = self.lstm(x)
        # output.shape: (max_seq_len, batch_size, hidden_size)

        # we discard output of encoder
        return hidden, cell

## Implement Decoder

The decoder is similar to the encoder in that it has an embedding layer that maps the target features (the indices associated with the characters in the target sequences) to vectors in a space of dimension that, ideally, is much less than the number of features, that is, the number of tokens.

The key difference is that the output of the LSTM is passed to a *Linear* layer that outputs a vector equal in size to the number of features, i.e., tokens associated with the target sequences. 

In [15]:
class Decoder(nn.Module):
    '''
    max_seq_len  maximum length of target sequences
    num_features number of target features, i.e., tokens
    '''
    def __init__(self, 
                 max_seq_len,
                 num_features, 
                 embed_size=10, 
                 hidden_size=HIDDEN_SIZE, 
                 num_layers=NUM_LAYERS, 
                 dropout=0.5, 
                 device=DEVICE):
        super(Decoder, self).__init__()
        
        self.max_seq_len = max_seq_len   # this is used below
        self.num_features= num_features  # number of target tokens
        self.embed_size  = embed_size
        self.hidden_size = hidden_size
        self.num_layers  = num_layers
        self.dropout     = dropout
        self.device      = device
 
        # The shape of inputs must be (max_seq_len, batch_size)
        # The shape of outputs is (max_seq_len, batch_size, embed_size)
        self.embedding   = nn.Embedding(num_features, 
                                        embed_size).to(device)

        # inputs.shape:  (max_seq_len, batch_size, embed_size)
        # outputs.shape: (max_seq_len, batch_size, hidden_size)
        self.lstm = nn.LSTM(input_size  = embed_size, 
                            hidden_size = hidden_size, 
                            num_layers  = num_layers,
                            dropout     = dropout).to(device)

        self.linear = nn.Linear(hidden_size, num_features).to(device)

    def forward(self, source, hidden, cell):
        # For a given batch instance, the indices of tokens will be 
        # passed one by one to this function. Therefore, the source
        # is a 1D tensor of token indices of shape (batch_size).
        # But since the embedding object requires an input with shape 
        # (max_seq_len, batch_size) we need to use unsqueeze(0) to 
        # insert an extra dimension of size 1 at dim=0 so that the
        # shape is (1, batch_size).
        
        source = source.unsqueeze(0).to(self.device)

        # source.shape:   (1, batch_size)
        
        # Map each index in the batch of indices to its embedded 
        # representation.
        embedded = self.embedding(source)
        # embedded.shape: (1, batch_size, embed_size)
                
        # Input embedded representation of token and previous
        # hidden and cell states to lstm
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output.shape:   (1, batch_size, hidden_size)
        
        # Get rid of dim 0 of length "1" to match shape expected by
        # the linear layer
        output = output.squeeze(0) 
        # output.shape:   (batch_size, hidden_size)
        
        prediction = self.linear(output)
        # prediction.shape: (batch_size, num_features)
        
        # We don't need to apply a softmax to prediction because 
        # this is done by the PyTorch cross entropy class. 
        return prediction, hidden, cell

### Construct seq2seq model

In [16]:
class Model(nn.Module):
    '''
    model = Model(encoder, decoder, device)
    '''
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device  = device
        self.max_seq_len  = self.decoder.max_seq_len
        self.num_features = self.decoder.num_features
        
        assert encoder.hidden_size == decoder.hidden_size, \
            "hidden_size of encoder and decoder must be equal!"
        
        assert encoder.num_layers == decoder.num_layers, \
            "num_layers of encoder and decoder must be equal!"
        
    def forward(self, source, target=None):
        '''
    1.  If target is a tuple, then it is interpreted as:
        target = (target, teacher_prob)
            
    2.  If the target is not a tuple, it is assumed to be the target
        and the teacher_prob is set to 0.5
        
    When the model is called in eval mode, only the source need be
    given, in which case teacher_prob is set to 0.0.
        '''

        source = source.to(self.device)
        if target == None:
            # the target is not used in eval mode
            teacher_prob = 0.0
        elif type(target) == type(()):
            target, teacher_prob = target
            target = target.to(self.device)
        else:
            teacher_prob = 0.5
            target = target.to(self.device)

        # (see definition of x_max_seq_len in Seq2SeqDataPreparer)
        # source.shape = (x_max_seq_len, batch_size)
        # target.shape = (y_max_seq_len, batch_size)
        #
        # teacher_prob is the probability, during training, to 
        # use the true target rather than the predicted target.
        # ideally that probability should be gradually reduced as the
        # training progresses.
           
        # Tensor to store decoder outputs
        batch_size = source.shape[1] # same for targets
        outputs = torch.zeros(self.max_seq_len, 
                              batch_size, 
                              self.num_features).to(self.device)
        
        # Use last hidden state of the encoder as the initial 
        # hidden state of the decoder. Note: the encoder discards
        # the output of the LSTM.
        hidden, cell = self.encoder(source)
        
        # Loop over the source indices corresponding to tokens.
        # The first input to the decoder should be the index 
        # associated with the tab token. 
        index = source[0,:]
        
        for t in range(1, self.max_seq_len):
            
            output, hidden, cell = self.decoder(index, hidden, cell)
            
            # Cache predictions for each token
            # output.shape: (batch_size, num_features)
            outputs[t] = output
                       
            # Get index of prediction with highest value
            prediction = output.argmax(1) 
            
            # For the next index, decide whether to use the
            # target or the prediction. Note: in eval mode, since
            # only the source is specified, teacher_prob will be 
            # zero (by construction) so that the predicted indices,
            # and therefore tokens, will be used.
            use_target = rn.random() < teacher_prob
            index      = target[t] if use_target else prediction
        
        # Note: For a given batch instance, the num_features outputs
        # do not sum to unity.
        return outputs

### Train Model

  1. Create an encoder
  1. Create a decoder
  1. Create a encoder/decoder model
  1. Train model

In [19]:
max_source_seq_len  = db.max_seq_len('source')
num_source_features = db.num_tokens('source')
encoder_embed_size  = 5
encoder_dropout     = 0.5

max_target_seq_len  = db.max_seq_len('target')
num_target_features = db.num_tokens('target')
decoder_embed_size  = 5
decoder_dropout     = 0.5

# in this model, the following structural parameters are the same for
# both the encoder and decoder. 
hidden_size         = HIDDEN_SIZE  # size of hidden and cell state vectors
num_layers          = NUM_LAYERS   # number of stacked LSTMs

encoder = Encoder(max_source_seq_len,
                  num_source_features, 
                  encoder_embed_size, 
                  hidden_size, 
                  num_layers, 
                  encoder_dropout).to(DEVICE)

decoder = Decoder(max_target_seq_len,
                  num_target_features, 
                  decoder_embed_size, 
                  hidden_size, 
                  num_layers, 
                  decoder_dropout).to(DEVICE)

model = Model(encoder, decoder, DEVICE).to(DEVICE)
print(model)
print('Computational device:      %s' % DEVICE)
print('Number of free parameters: %d' % count_parameters(model))

Model(
  (encoder): Encoder(
    (embedding): Embedding(31, 5)
    (lstm): LSTM(5, 32, num_layers=10, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(35, 5)
    (lstm): LSTM(5, 32, num_layers=10, dropout=0.5)
    (linear): Linear(in_features=32, out_features=35, bias=True)
  )
)
Computational device:      cpu
Number of free parameters: 163533


### Choose optimizer

In [20]:
optimizer = optim.Adam(model.parameters())

### Choose loss function
For each token with associated index k, compute the cross-entropy loss

\begin{align*}
    E_k & = -\log \left( \frac{\exp(\hat{y}_k)}{\sum_{\{j\}} \exp(\hat{y}_j) } \right) ,
\end{align*}

where $\hat{y}_j$ is the $j^\text{th}$ element of the model's output vector of length *num_features*. The losses $E_k$ are averaged over all tokens in a sequence and all batch instances. The set $\{ j \}$  excludes tokens that correspond to padding. 

In [21]:
space = db.x_token2index[' ']
criterion = nn.CrossEntropyLoss(ignore_index=space).to(DEVICE)

### Define trainer

In [24]:
def train(model, dataloader, optimizer, criterion, device, clip):
    
    # set to train mode
    model.train()  
    
    epoch_loss  = 0
    count = 0
    
    # Important: reset dataloader to recreate the iterator.
    # This is necessary because Python's iter(*) class has no means
    # to reset it.
    dataloader.reset() 
    
    for i, (X, Y) in enumerate(dataloader):
        #print('X.shape:', X.shape, ' Y.shape:', Y.shape)
        
        # zero gradients of all trainable parameters
        optimizer.zero_grad()
        
        # the dataloader has been initialized with pinned memory so
        # that, in principle, the transfer to the device is faster.
        X = X.to(device)
        Y = Y.to(device)
        
        # compute output of model
        output = model(X, Y)
        # output.shape: (max_target_seq_len, batch_size, num_features)
        #print('output.shape(before):', output.shape)
    
        num_features = output.shape[-1]
        
        # skip first token (a tab), then reshape to
        # ((max_target_seq_len-1)*batch_size, num_features)
        output = output[1:].reshape((-1, num_features))
        #print('output.shape(after):', output.shape)

        # skip first token (a tab), then reshape to
        # ((max_target_seq_len-1)*batch_size)
        Y = Y[1:].reshape(-1)
               
        # average loss over mini-batches
        loss = criterion(output, Y)
        
        # compute gradients using automatic differentiation
        loss.backward()
        
        # clip gradients so they don't blow up
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # make one step in th model parameter space
        optimizer.step()
        
        epoch_loss += loss.item() # to CPU, if using a GPU
        count += 1
        
        if i % 10 == 0:
            print('%d ' % i, end='')
            
    print('%d end epoch' % count)
    return epoch_loss / count

### Evaluator

In evaluation mode, we turn off gradient calculation and we provide only the source sequences.

In [23]:
def evaluate(model, dataloader, criterion, device):
    
    model.eval()
    
    epoch_loss = 0
    count = 0
    
    # no need to compute gradients
    with torch.no_grad():
    
        # Important: reset dataloader to recreate iterator
        dataloader.reset()

        for i, (X, Y) in enumerate(dataloader):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            # output.shape:(max_target_seq_len,batch_size,num_features)

            num_features = output.shape[-1]
        
            # skip first token (a tab), then reshape to
            # ((max_target_seq_len-1)*batch_size, num_features)
            output = output[1:].reshape((-1, num_features))
                            
            # skip first token (a tab), then reshape targets to
            # ((max_target_seq_len-1)*batch_size)
            Y = Y[1:].reshape(-1)
               
            # average loss over mini-batches, excluding padding tokens
            loss = criterion(output, Y)
        
            epoch_loss += loss.item()
            count += 1
            
    return epoch_loss / count

### Finally, train!


Note: The class __Seq2SeqDataLoader__ is a custom dataloader that returns batches of sequence pairs comprising sources and targets, X and Y, respectively. The quantities X and Y are 2D tensors, each with shape (max_seq_len, batch_size) that comprise sequences of indices arranged in columns. Each index corresponds to a token, i.e., a character. The column lengths, max_seq_len, differ between X and Y.


__NB:__ At present, Python iterators can't be reset, so we need to remake them each time to ensure each starts at the beginning. We do so by calling the __reset()__ method of __Seq2SeqDataLoader__.

In [50]:
#=================================================================
BATCH_SIZE = 50      # number of instances per batch
N_EPOCHS   = 10      # number of times through training data
CLIP       = 1       # prevent gradients from exploding!
#=================================================================
train_dataloader = Seq2SeqDataLoader(db.x_train, db.y_train, BATCH_SIZE)
valid_dataloader = Seq2SeqDataLoader(db.x_valid, db.y_valid, BATCH_SIZE)
test_dataloader  = Seq2SeqDataLoader(db.x_test,  db.y_test,  BATCH_SIZE)

In [26]:
def train_me():
    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):
        print('epoch: %5d' % epoch)
    
        start_time = time.time()
    
        train_loss = train(model, train_dataloader, optimizer, 
                           criterion, DEVICE, CLIP)
    
        valid_loss = evaluate(model, valid_dataloader, 
                              criterion, DEVICE)
    
        end_time   = time.time()
        mins, secs = epoch_time(start_time, end_time)
    
        if valid_loss < best_valid_loss:
            # save best model so far
            best_valid_loss = valid_loss
            torch.save(model.state_dict(),
                       '%s/seq2seq_model.pt' % BASE)
    
        print(' duration(mm:ss): %2.2d:%2.2d | train_loss: %7.4f |'\
              ' valid_loss: %7.4f\n' % \
              (mins, secs, train_loss, valid_loss))
    print('\ndone!')
    
train_me()

### Now test!

In [27]:
model.load_state_dict(torch.load('%s/seq2seq_model.pt' % BASE))
test_loss = evaluate(model, test_dataloader, criterion, DEVICE)
print('test loss: %7.4f' % test_loss)